# Check Constraints

Una restricción `CHECK` permite especificar una condición sobre el valor de una fila.
La sintaxis es:

``` 
CREATE TABLE table_name (
  column1 datatype [ NULL | NOT NULL ],
  column2 datatype [ NULL | NOT NULL ],

  ...

  CONSTRAINT constraint_name
    CHECK [ NOT FOR REPLICATION ] (column_name condition)

);
```
*table_name*       El nombre de la tabla a la cual le agregaremos la restricción `CHECK`.<br>
*constraint_name* El nombre que le asignaremos a la restricción. <br>
*column_name* La columna a la cual se le aplica la restricción.<br>
*condition* La condicion que se debe cumplir.



In [1]:
--Para dejar en blanco la base de datos prueba ejecute este bloque
--Ejecute este bloque(colapsado) para usarla 

IF EXISTS 
   (
     SELECT name FROM master.dbo.sysdatabases 
     WHERE name = 'Pruebas'
    )
    DROP DATABASE Pruebas
GO

CREATE DATABASE Pruebas
GO

use Pruebas


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.481

Por ejemplo el salario de un empleado debe ser mayor a 0, por lo tanto a esa columna le ponemos esta restricción

In [2]:
CREATE TABLE Empleados
( 
    idEmpleado INT NOT NULL,
    apellido VARCHAR(50) NOT NULL,
    nombre VARCHAR(50),
    salario numeric(10,3),
    CONSTRAINT check_salario
        CHECK (salario > 0)
);


Commands completed successfully.

Total execution time: 00:00:00.004

¿Que pasa si agregamos a un empleado con un salario menor o igual a cero?
Error al insertar ya que existe una restriccion que no acepte cantidades igual o menor a cero


In [3]:
INSERT  empleados
    (idEmpleado,  nombre, apellido,salario)
    VALUES (1, 'Rosario', 'Uzárraga', 0 )

: Msg 547, Level 16, State 0, Line 1
The INSERT statement conflicted with the CHECK constraint "check_salario". The conflict occurred in database "Pruebas", table "dbo.Empleados", column 'salario'.

The statement has been terminated.

Total execution time: 00:00:00.007

Agregue una restricción para que la columna sexo solo permita los valores 'M' o 'F'

In [4]:
ALTER TABLE empleados add sexo CHAR(1), CONSTRAINT chk_sexo CHECK (sexo in ('M','F')) 

Commands completed successfully.

Total execution time: 00:00:00.004

In [5]:
-- Agregue una fila que cumpla la condición
insert empleados VALUES (2,'Karla','Murillo',1500,'F')

(1 row affected)

Total execution time: 00:00:00.002

In [7]:
-- Agregue una fila que NO cumpla la condición
insert empleados VALUES (3,'Estela','Mendez',1500,'T')

: Msg 547, Level 16, State 0, Line 2
The INSERT statement conflicted with the CHECK constraint "chk_sexo". The conflict occurred in database "Pruebas", table "dbo.Empleados", column 'sexo'.

The statement has been terminated.

Total execution time: 00:00:00

Agregue dos columnas: `nacimiento` y `contratación` donde la fecha de contratación no debe ser anterior al nacimiento

In [8]:
ALTER TABLE empleados add nacimiento DATE, contratacion DATE, CONSTRAINT chk_fecha CHECK (contratacion > nacimiento)

Commands completed successfully.

Total execution time: 00:00:00.001

In [12]:
-- Agregue una fila que cumpla la condición
update empleados 
set nacimiento = '19990505', contratacion = '20200211'
where idEmpleado = 2

(1 row affected)

Total execution time: 00:00:00

In [19]:
select * from empleados

(2 rows affected)

Total execution time: 00:00:00.003

idEmpleado,apellido,nombre,salario,sexo,nacimiento,contratacion
2,Karla,Murillo,1500.000,F,1999-05-05,2020-02-11
3,Estela,Mendez,1500.000,M,NULL,NULL


In [14]:
-- Agregue una fila que NO cumpla la condición
update empleados 
set nacimiento = '19990915', contratacion = '19910211'
where idEmpleado = 3

: Msg 547, Level 16, State 0, Line 2
The UPDATE statement conflicted with the CHECK constraint "chk_fecha". The conflict occurred in database "Pruebas", table "dbo.Empleados".

The statement has been terminated.

Total execution time: 00:00:00

La fecha de nacimiento del empleado no puede ser posterior al la fecha actual. Utilice la funcion del sistema `GETDATE()` para definir la condición.

In [20]:
ALTER TABLE empleados add CONSTRAINT chk_fecha2 CHECK (nacimiento < getDate())

Commands completed successfully.

Total execution time: 00:00:00.003

In [22]:
-- Agregue una fila que cumpla la condición
INSERT  empleados
    VALUES (1, 'Mayra', 'Marquez', 9000, 'M', '20000211','20220212')

(1 row affected)

Total execution time: 00:00:00

In [24]:
-- Agregue una fila que NO cumpla la condición
INSERT  empleados
    VALUES (4, 'Saul', 'Sanz', 8000, 'M', '20230211','20220212')

: Msg 547, Level 16, State 0, Line 2
The INSERT statement conflicted with the CHECK constraint "chk_fecha". The conflict occurred in database "Pruebas", table "dbo.Empleados".

The statement has been terminated.

Total execution time: 00:00:00

In [23]:
select * from empleados

(3 rows affected)

Total execution time: 00:00:00.003

idEmpleado,apellido,nombre,salario,sexo,nacimiento,contratacion
2,Karla,Murillo,1500.000,F,1999-05-05,2020-02-11
3,Estela,Mendez,1500.000,M,NULL,NULL
1,Mayra,Marquez,9000.000,M,2000-02-11,2022-02-12


De hecho la empresa no puede contratar a menores de edad. Redefina la condición, eliminando la anterior y creandola otra vez

In [31]:
alter table empleados drop CONSTRAINT chk_fecha
alter table empleados add CONSTRAINT chk_fecha CHECK (DATEDIFF(year,nacimiento,getdate()) > 18 and nacimiento < contratacion )

Commands completed successfully.

Total execution time: 00:00:00.003

In [32]:
-- Agregue una fila que cumpla la condición
INSERT  empleados
    VALUES (5, 'Mauricio', 'Garcia', 20000, 'M', '19990915','20200212')

(1 row affected)

Total execution time: 00:00:00.003

In [33]:
-- Agregue una fila que NO cumpla la condición
INSERT  empleados
    VALUES (5, 'Mauricio', 'Garcia', 20000, 'M', '20050915','20200212')

: Msg 547, Level 16, State 0, Line 2
The INSERT statement conflicted with the CHECK constraint "chk_fecha". The conflict occurred in database "Pruebas", table "dbo.Empleados".

The statement has been terminated.

Total execution time: 00:00:00